# Working with global data
*GEOL 202A | Lab 6 | Prof. Lizz Ultee, with thanks to [Fabien Maussion](https://fabienmaussion.info/climate_system/week_03/01_Lesson_MoreDataCrunching.html)*

How can we extract useful information from a dataset we manage to find?  

Here we are considering global atmospheric data, but you might also have large datasets from ice sheets, oceans, or geological observations.  It is useful to have some techniques to visualize, process, and extract information from these large datasets.

In this lab, we will use **pair programming** to manipulate a global dataset.  You and your partner will work through example data processing on global 2-meter air temperature, then apply your skills to explore a new dataset.  Start off with one partner "driving" (typing the code) and one "navigating" (suggesting what to write).  Switch navigator/driver roles every time you see a new number.

In [ ]:
from IPython.display import IFrame ## Prof. Ultee using this to add embedded puzzles

## 1. Temperature data  

First, download the global 2-meter air temperature dataset `ERA5_LowRes_Monthly_t2m.nc` from [this page](https://fabienmaussion.info/climate_system/download.html).  While you are there, download `ERA5_LowRes_Invariant.nc` -- we will use it later.

Place both files in an easily accessible folder.  The cell below assumes you have them in a folder called `data` one level above this Jupyter notebook.

Import the packages we will use, and open the dataset.

In [1]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
ds = xr.open_dataset('../data/ERA5_LowRes_Monthly_t2m.nc')
ds

**Q: What is ERA5? Search the internet to get some basic information about it. What institution provides the data? What time period is it available for?**

*your notes here*

**Q: How is the ERA5 data produced?  How is "reanalysis" data different from observations?**

*your notes here*

Use the "HTML representation" of the dataset printed above to figure out what to expect from the data:
- **what are the dimensions of the t2m variable?**
- **what is the spatial resolution of this dataset?**
- **what is its temporal resolution?**
- **what it the data type of the various variables and coordinates available in the dataset? What are the differences between each data type?**

## A map plot of average air temperature

Let's compute the time average of the air temperature. 

We will try something new here: I have set up a puzzle for you to drag the lines of code into the correct order.  Don't edit the `IFrame` command below.  Just run it, then click and drag the lines to the solution side.  You can click "Get feedback" to check whether your setup is correct.

Once you have a correct solution, copy the lines of code into a new cell to generate the map plot.

In [2]:
IFrame('http://parsons.problemsolving.io/puzzle/7471c347a9294eb580802df26a2cd471', width=1000, height=400)


In [ ]:
## your code here

### 2. Adding discrete levels

Smooth (continuous) color maps like the above "look nice", but the human eye is not trained to see such small differences in color. For example, it would be quite difficult to tell the temperature of the Peruvian coast (above 280K? or below?). Sometimes, **discrete levels** are the way to go:  

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels=[240, 260, 280, 285, 290, 295, 300]) 
ax.coastlines(); ax.gridlines(); 

**Q: in which range can we place the temperature off the Peruvian coast? If your eyes still can´t make out the difference, how can we make sure?**

*your response here*

**Exercise: Make a new plot, but this time set `levels=12`.  Explore how many levels are required to display the data most effectively.**

In [ ]:
# your code here

### Color tables

Let's make a new variable called ``t2c_tavg``, which is ``t2_tavg`` converted to degrees celsius:

In [ ]:
t2c_tavg = t2_tavg - 273.15

Copy your plotting code from above and use it to plot `t2c_tavg` instead of `t2_tavg`.

In [ ]:
## your code

What happened to the plot? Note the location of the 0 on the colorbar and the automated choice of a new colorscale. Note also that the data range is mostly dictated by very cold temperatures in Antarctica. The automated choices are a good first indication, but not always the most meaningful representation of the data. 

Let's see what happens with the below:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2c_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='inferno', center=False, 
              vmin=-40, vmax=20, levels=7, cbar_kwargs={'label': '°C'}) 
ax.set_title('Average annual 2m air temperature, ERA5 1979-2018')
ax.coastlines(); ax.gridlines(); 

**Q: try to understand the role of each keyword in the `plot` command by trying to use each of them separately. If you're still unsure, a look at xarray's [documentation](http://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html) might be helpful.**

Note: a list of matplotlib's color tables can be found [here](http://www.matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
# your workspace here

### Slightly faster map plots

xarray's `.plot` method internally uses matplotlib's [pcolormesh](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html) which, for reasons too long to explain here, is the more accurate way to represent gridded data on a map. If you are willing to sacrifice some accuracy (not visible with the bare eye at the global scale), you can also use [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html):

In [ ]:
t2_tavg = ds.t2m.mean(dim='time')
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot.imshow(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();

This plot should render about 4 times faster than the default plot, which is useful for data exploration. It should not be used for final rendering or for regional plots, though.

## 3. Extracting time series

**Exercise: print the dataset's time coordinate. Which period do the data cover?**

In [ ]:
# your answer here

Remember the selection methods we learned in Lab 2?  We can also use them for time series.  Let's define the variable `t2` for easy access and make a selection.

In [ ]:
t2 = ds.t2m
t2.sel(time='2008')

**Q: what did we just do? Does that make sense to you?**

*your notes here*

The selection methods are very flexible.  **Make a prediction: what do you think each of the following commands will do?  Test them out and see.**
- t2.sel(time='2008-02')
- t2.sel(time='2008/02')
- t2.sel(time=slice('2008', '2012'))

In [ ]:
# Your answer here

### Time series of globally averaged fields 

Do you remember the average air temperature on Earth?  I know you do.  Let's see if we can extract a global average from this ERA5 data and compare it with our prior knowledge.

Recall that to compute a global average on a sphere, we first need to weight the data by latitude (go back to [Lab 2](https://github.com/ehultee/climdyn-labs/blob/main/02-FM-NetCDF_Data.ipynb) if you need a refresher).

In [ ]:
# Meridional weights
weight = np.cos(np.deg2rad(ds.latitude))
weight = weight / weight.sum()

# Meridionally weighted zonal mean
zonal_mean_t2_c = t2.mean(dim='longitude') - 273.15  # convert into Celsius
weighted_zonal_mean_t2_c = zonal_mean_t2_c * weight
weighted_ts_t2_c = weighted_zonal_mean_t2_c.sum(dim='latitude')

Notice the second to last line in the cell above.  Although `zonal_mean_t2_c` is of dimensions (`time`, `latitude`), we successfully multiplied it by an array of dimension `latitude` only. This is called "broadcasting" in the numpy jargon, and something you don't have to worry about for now: xarray does it for you! Indeed, xarray knows about the dimensions of your data (and their coordinates) and will always make arithmetic operations match these.

Let's examine the average surface temperature over the time period of the data.

In [ ]:
weighted_ts_t2_c.plot();

Logically, the global average temperature on Earth would be: 

In [ ]:
weighted_ts_t2_c.mean(dim='time')

**Q: Does the global average reflect what we discussed in previous weeks?**  
Questions like these are called "sanity checks" and help us confirm our code is producing sensible answers.

In [ ]:
# your answer here

### Resampling time series data

Resampling is the operation of changing the *sampling* of the data, i.e. the frequency at which it is sampled. One of the most meaningful ways to resample is to do an average, for example over each year:

In [ ]:
tsa_t2_c = weighted_ts_t2_c.resample(time='AS').mean()

**Exercise: check that ``tsa_t2_c`` is what you think it is. What could the ``'AS'`` mean? Try ``'A'`` for a change, and note the difference.  What happens if you choose `MS`?**

Note that averaging is [not the only way](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html) available to resample data. **Try ``.std()`` and  ``.max()``, too.**

In [ ]:
# your answer here

**Exercise: Make a yearly average plot from `tsa_t2_c`. Describe its features.  Does it surprise you, or is it the way you thought it would be? Compute the standard deviation of this time series (tip: .std())** 

In [ ]:
# your answer here

### Computing a monthly climatography (or annual cycle)

Another way to look at time series data is to average them according to the time of year to study the annual cycle. This is done with the ``.groupby()`` method:

In [ ]:
tsm_t2_c = weighted_ts_t2_c.groupby('time.month').mean()

**Exercise: check that `tsm_t2_c` is what you think it is. What is the new dimension of the data? Plot it. Can you explain what you see?**

In [ ]:
# your answer here

### Averages and anomalies 

Remember when you found the temperature anomaly of Middlebury observations compared with a long-term average?  We can decompose any time series $A(t)$ into a reference value (often the mean) $\overline{A}$ and an anomaly $A'(t)$:

$$A(t) = \overline{A} + A'(t)$$

Often the variable t is omitted from the notation, i.e. one writes $A = \overline{A} + A'$.

**Exercise: compute the global temperature anomaly for the year 1997 with respect to the 1979-2018 average.**
This is a multi-step task; you and your partner may want to write out a plan on paper first.

In [ ]:
# your answer here

## 4. Selecting specific areas of our data

In the section above, we selected snapshots in time to generate time series.  We can also select spatial slices out of the data:

In [ ]:
t2_reg = t2_tavg.sel(longitude=slice(-40, 40))

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree()) # Note that I changed the projection
t2_reg.plot(ax=ax, transform=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.BORDERS); # What does this command do? 
ax.coastlines();

**Exercise: create a new "t2c_reg" variable which is a subset of t2c_tavg between the longitudes (-20, 60) and the latitudes (40, -40). Plot the result.** (*note: yes, it is (40, -40) and not (-40, 40)*)

In [ ]:
# your answer here

### Selection based on a condition

What if we are interested into air temperature on land only, and want to remove the oceans from our analyses? For this we can use a technique called "masking".  We will download a file with the same spatial resolution as our data, but with the core purpose of identifying which areas are land and which are ocean. 

First, load in the "Invariant" data.

In [ ]:
nc_inv = xr.open_dataset('../data/ERA5_LowRes_Invariant.nc')
nc_inv

**Exercise: explore this new data file (variables, units). Plot the variables.**

In [ ]:
# your answer here

OK, so in the `lsm` variable "1" is land, "0" is ocean. We are going to use this information to mask out the values from the ocean:

In [ ]:
masked_t2_avg = t2c_tavg.where(nc_inv.lsm > 0.5)

**Exercise: Plot this new variable on a map. Then compute its zonal average and plot it!**

In [ ]:
# your answer here

# (decide on your own coding plan) Exploring a new dataset
Choose a different one of the pre-processed ERA5 datasets linked on [this page](https://fabienmaussion.info/climate_system/download.html) and download it.  I suggest you choose one of the "Monthly surface (3D)" sets.  With your partner, make a plan to explore this dataset.  You might want to draft your plan on paper before starting to code.  At minimum, the code you write below should allow you to:
- Load in the data
- Identify what variable(s) are included in the dataset
- Highlight an interesting pattern or feature in the data 
- Define (and ideally answer) a science question based on the data

Remember to include text cells describing the approach and interpretation, and comments in the code to aid readability.